In [1]:
# %matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns



import sys
sys.path.append("./../scripts")
import PandemiXFunctions as pf

import geopandas as gpd

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams['figure.facecolor'] = (1,1,1,1)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

# from datetime import date
import datetime


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

saveFigures is set to: True
Done loading packages


In [2]:
import mpld3

In [3]:
# Define paths
rootdir_data = os.getcwd() +"/../DanskeData/" 

path_data = rootdir_data + "ssi_data/"
path_dash = rootdir_data + "ssi_dashboard/"
path_vacc = rootdir_data + "ssi_vacc/"

path_figs = os.getcwd() +"/../Figures/" 

In [4]:

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir

dfMain = pd.read_csv(latestdir+'/Regionalt_DB/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv',encoding='latin-1',delimiter=';')

dfMain = dfMain.rename({'Prøvetagningsdato':'Date','Døde':'Deaths','Bekræftede tilfælde i alt':'Cases'},axis=1)
dfMain['Date'] = pd.to_datetime(dfMain.Date)
# dfMain
dfSum = dfMain.groupby('Date').sum()

# Make a figure of cases and deatsh

In [5]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
# fig,(ax1,ax2) = plt.subplots(2,1)

ax1.plot(dfSum.index,dfSum.Cases,'k.:',markersize=1,linewidth=0.5,label='Data')
ax1.plot(pf.rnTime(dfSum.index),pf.rnMean(dfSum.Cases),'k',label='7-dages gennemsnit')
ax2.plot(dfSum.index,dfSum.Deaths,'k.:',markersize=1,linewidth=0.5,label='Data')
ax2.plot(pf.rnTime(dfSum.index),pf.rnMean(dfSum.Deaths),'k',label='7-dages gennemsnit')
# ax3.plot(dfSum.index,dfSum.Cases,'k.:',markersize=1,linewidth=0.5,label='Data')
# ax3.plot(pf.rnTime(dfSum.index),pf.rnMean(dfSum.Cases),'k',label='7-dages gennemsnit')

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
# ax3.set_ylim(bottom=0,top=5000)

ax1.set_xlim(left=dfSum.index[0])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.%b\n%Y'))

ax1.set_ylabel('Smittetilfælde')
ax2.set_ylabel('Dødsfald')

fig.tight_layout()

mpld3.display()

In [6]:
from mpld3 import plugins 


In [19]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(dfSum.index,dfSum.Cases,'k.:',markersize=1,linewidth=0.5,label='Data')
ax1.plot(pf.rnTime(dfSum.index),pf.rnMean(dfSum.Cases),'k',label='7-dages gennemsnit')
ax2.plot(dfSum.index,dfSum.Deaths,'k.:',markersize=1,linewidth=0.5,label='Data')
ax2.plot(pf.rnTime(dfSum.index),pf.rnMean(dfSum.Deaths),'k',label='7-dages gennemsnit')

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)

ax1.set_xlim(left=dfSum.index[0])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.%b\n%Y'))

ax1.set_ylabel('Smittetilfælde')
ax2.set_ylabel('Dødsfald')

fig.tight_layout()
# plugins.clear(fig)  # clear all plugins from the figure

# plugins.connect(fig, plugins.Reset(),  plugins.Zoom(),  
#                 plugins.ConfigurableZoomAndPan(
#                     ax2, zoom_y=False, xlim=(-1,15), ylim=(-1,10))
#                 )

mpld3.display()
# mpld3.fig_to_html(fig)

# Gennembrudsfiler

In [8]:
# Start by getting the most recent data

for k in range(0,7):
    dayToCheck = np.datetime64('today')-np.timedelta64(k,'D')
    thisWeekDay = (dayToCheck).astype(datetime.datetime).isoweekday()    
    if (thisWeekDay == 2):
        tuePath = 'SSI_data_'+str(dayToCheck)
        if (os.path.exists(path_data + tuePath) == False):
            print('No data for most recent tuesday... If after 14:00, download new data')
            tuePath = 'SSI_data_'+str(dayToCheck-np.timedelta64(7,'D')) 

print(f'Path to latest Tuesday was {tuePath}')

dirPath = path_data + tuePath + '/'

dfRecent = pd.read_csv(dirPath+'gennembrudsinfektioner_table1.csv',delimiter=';')
df2_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_cases.csv',delimiter=';')
df2_D = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_dode.csv',delimiter=';')
df3_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_cases.csv',delimiter=';')
df3_A = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_alle.csv',delimiter=';')

df2_T = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_tests.csv',delimiter=';')
df3_T = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_tests.csv',delimiter=';')
# df3_C.iloc[:,1:] = pd.to_numeric(df3_C.iloc[:,1:].str.replace(',','.'))
for col in df3_C.columns[1:]:
    df3_C[col] = pd.to_numeric(df3_C[col].str.replace(',','.'))
    df3_A[col] = pd.to_numeric(df3_A[col].str.replace(',','.'))
    df3_T[col] = pd.to_numeric(df3_T[col].str.replace(',','.'))


Path to latest Tuesday was SSI_data_2022-04-05


In [9]:
posWeekNames = np.unique([x[:11] for x in df2_C.columns[1:]])
weekOrder = np.argsort([x[-4:] + x[4:6] for x in posWeekNames])
allWeekNames = posWeekNames[weekOrder]

        
sNone = 'Ingen vaccination'        
sOne = 'Første vaccination'  
sTwo = 'Anden vaccination'
sFull = 'Fuld effekt efter primært forløb'   
sReva = 'Fuld effekt efter revaccination'

curDTs= [np.datetime64(datetime.datetime.strptime(d[-4:] + '-W'+d[4:6]+'-1', "%Y-W%W-%w")) for d in allWeekNames]


In [10]:
def getTimeSeries(thisdf=df2_C,curStatus='Ingen vaccination',curAge='Alle',weekNames=allWeekNames):
    # If curAge is just a string, return the given array
    if (type(curAge)==str):
        agedf = thisdf[thisdf.Aldersgruppe==curAge]
        allVals = []
        for curWeek in weekNames:
            toAdd = agedf[curWeek+'_'+curStatus].values[0]
            allVals.append(toAdd)
        allVals = np.array(allVals)
        
        return allVals 
        
    else:
        # If curAge is not a string, it is assumed to be a list of strings
        totCount = []
        for thisAge in curAge:
            agedf = thisdf[thisdf.Aldersgruppe==thisAge]
            allVals = []
            for curWeek in weekNames:
                toAdd = agedf[curWeek+'_'+curStatus].values[0]
                allVals.append(toAdd)
            allVals = np.array(allVals)
            if len(totCount) == 0:
                totCount = allVals 
            else:
                totCount = totCount + allVals
        return totCount

def getTimeSeriesAll(thisdf=df2_C,curAge='Alle',weekNames=allWeekNames):
    return getTimeSeries(thisdf,sNone,curAge,weekNames)+getTimeSeries(thisdf,sOne,curAge,weekNames)+getTimeSeries(thisdf,sTwo,curAge,weekNames)

getTimeSeriesAll()

array([ 73205,  76379, 119339, 126661, 171544, 266970, 280602, 279817,
       297395, 225845, 143491, 104605,  75971,  48100,  33348,  21505],
      dtype=int64)

# Make a figure of deaths and cases

In [11]:

# %matplotlib widget

In [12]:
fig,ax = plt.subplots()

ax.plot(curDTs,getTimeSeriesAll(df2_C))

mpld3.display()
